# Analisis de data EldenRing

Este proyecto explora datos detallados sobre Elden Ring (FromSoftware, 2022), analizando jefes, armas y stats de armas. El objetivo es revelar patrones ocultos, relaciones entre armas y dmg.

Datasets Utilizados 

1. Ultimate Elden Ring with Shadow of The Erdtree DLC
Creado por: Pedro Altobelli
Enlace: https://www.kaggle.com/datasets/pedroaltobelli/ultimate-elden-ring-with-shadow-of-the-erdtree-dlc
Licencia: CC0: Public Domain (libre para uso comercial, sin atribución requerida pero ética).
Contenido: Stats de jefes, armas, NPCs, ubicaciones y datos del DLC.

2. Elden Ring Ultimate Dataset
Creado por: Rob Mulla
Enlace: https://www.kaggle.com/datasets/robikscube/elden-ring-ultimate-dataset
Licencia: CC0: Public Domain (libre para uso comercial, sin atribución requerida pero ética).
Contenido: Armas, escudos, ubicaciones, etc. 

3. Los datos de debilidades y coordenadas de los jefes fueron extraídos del sitio:
[Fextralife Elden Ring Wiki](https://eldenring.wiki.fextralife.com/)  para fines educativos.
Crédito a Fextralife por la información recopilada. Por respeto no se compartiran la informacion recopilada. 

Este análisis no sería posible sin el trabajo de recopilación de datos de la comunidad. Por favor, considera citar los datasets originales si usas este repositorio:

Herramientas utilizadas
Lenguajes: Python (pandas, matplotlib, seaborn).
Repositorio: GitHub.

Hallazgos Clave 
Peso vs tipo de dmg, Poder de ataque, debilidades de bosses, regiones donde hay gran concentracion de bosses y criaturas.
Armas más efectivas: Relación entre debilidades de bosses y el dmg del arma.

Las imágenes/frases de Elden Ring son propiedad de FromSoftware/Bandai Namco. Este proyecto es solo con fines educativos.
Contribuciones: ¡Siéntete libre de hacer un fork o sugerir mejoras!

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast 
from ast import literal_eval
import os
from pathlib import Path
from pandas import json_normalize
import re
from collections import defaultdict
from IPython.display import Image, display
import numpy as np 
from collections.abc import Iterable
import json
from warnings import simplefilter
from PIL import Image
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import matplotlib.image as mpimg

In [2]:
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_rows', None)  

Se importan los csv con las tablas que se utilizaran en el analisis, se agregaran como tablas en un diccionario "dataframes" para poder acceder a ellas facilmente

In [3]:
carpeta = Path(r"C:\Users\sayur\OneDrive\Documentos\DataSets\eldenring2")
dataframes = {}  #Nombre del diccionario
for archivo in carpeta.glob('*.csv'):
    nombre_tabla = archivo.stem  
    dataframes[nombre_tabla] = pd.read_csv(archivo,dtype=str, low_memory=False, na_filter=True)

La función "expand_list_of_dicts" transforma una columna en un DataFrame(tabla) de pandas que contiene listas/listas anidadas de diccionarios en sus propias columnas individuales basadas en las claves de los diccionarios.

In [4]:
def expand_list_of_dicts(df, column_name, primary_key='auto', exclude_keys=None):
    """
    Expande columnas que contienen listas de diccionarios.
    Ejemplo: [{'name': 'attack', 'amount': 10}, {'name': 'defense', 'amount': 5}]
    """
    if exclude_keys is None:
        exclude_keys = []
    
    try:
        data = df[column_name].apply(lambda x: ast.literal_eval(x) if pd.notna(x) and isinstance(x, str) else [])
    except (ValueError, SyntaxError) as e:
        print(f"⚠️ Error al procesar {column_name}: {str(e)}")
        return df
    
    all_keys = set()
    for item in data.explode():
        if isinstance(item, dict):
            all_keys.update(k for k in item.keys() if k not in exclude_keys)
    
    if not all_keys:
        return df
    
    if primary_key == 'auto':
        primary_key = 'name' if 'name' in all_keys else next(iter(all_keys))
    elif primary_key not in all_keys:
        primary_key = next(iter(all_keys))
    
    value_keys = [k for k in all_keys if k != primary_key]
    new_columns = defaultdict(list)
    
    for idx, items in enumerate(data):
        temp = {}
        if isinstance(items, list):
            for item in items:
                if isinstance(item, dict):
                    p_value = item.get(primary_key)
                    if p_value:
                        for v_key in value_keys:
                            col_name = f"{column_name}_{p_value}_{v_key}"
                            temp[col_name] = item.get(v_key)
        
        for col in new_columns:
            new_columns[col].append(temp.get(col))
        for col in temp:
            if col not in new_columns:
                new_columns[col] = [None] * idx + [temp[col]]
    
    for col_name, values in new_columns.items():
        df[col_name] = values
    
    return df

La función "expand_direct_dicts" maneja  las columnas con diccionarios directos

In [5]:
def expand_direct_dicts(df, column_name):
    """
    Expande columnas que contienen diccionarios directamente.
    Ejemplo: {'attack': 10, 'defense': 5}
    """
    try:
        # Convertir strings a diccionarios si es necesario
        data = df[column_name].apply(lambda x: ast.literal_eval(x) if pd.notna(x) and isinstance(x, str) else {})
        
        # Obtener todas las claves únicas de los diccionarios
        all_keys = set()
        for d in data:
            if isinstance(d, dict):
                all_keys.update(d.keys())
        
        # Crear nuevas columnas para cada clave
        for key in all_keys:
            new_col_name = f"{column_name}_{key}"
            df[new_col_name] = data.apply(lambda x: x.get(key) if isinstance(x, dict) else None)
        
        return df
    
    except (ValueError, SyntaxError) as e:
        print(f"⚠️ Error al expandir diccionarios en {column_name}: {str(e)}")
        return df

La función "smart_expand_columns" detecta automáticamente el formato de datos que hay en una columna de un DataFrame(tabla) y decide cómo expandirlo correctamente mandando a llamar las dos funciones anteriores

In [6]:
def smart_expand_columns(df, column_name):
    """
    Decide automáticamente cómo expandir la columna.
    """
    if df[column_name].empty:
        return df
    
    sample = df[column_name].dropna().iloc[0] if not df[column_name].dropna().empty else ""
    
    if isinstance(sample, str):
        try:
            parsed = ast.literal_eval(sample)
            if isinstance(parsed, list) and parsed and isinstance(parsed[0], dict):
                return expand_list_of_dicts(df, column_name)
            elif isinstance(parsed, dict):
                return expand_direct_dicts(df, column_name)
        except (ValueError, SyntaxError):
            pass
    
    return df

La funcion "normalize_all_dataframes" automatiza el proceso de normalización estructural para todos los DataFrames en el diccionario. 
Transformar todas las columnas anidadas (listas de diccionarios o diccionarios directos) en columnas planas en el DataFrames.

In [7]:
def normalize_all_dataframes(dataframes_dict):
    for table_name, df in dataframes_dict.items():
        for col in df.columns:
            if df[col].empty:
                continue
            df = smart_expand_columns(df, col)  # <- Ahora llama a la versión singular
        dataframes_dict[table_name] = df
    return dataframes_dict
    # Aplicar a todos los DataFrames
dataframes = normalize_all_dataframes(dataframes)
# Verificar resultados
for name, df in dataframes.items():
    print(f"📊 Tabla: {name}")
    #print("Nuevas columnas:", [c for c in df.columns if c != name])

📊 Tabla: bosses
📊 Tabla: bosses_final_clean
📊 Tabla: creatures
📊 Tabla: locations
📊 Tabla: locations_coords
📊 Tabla: weapons2


Con la funcion "convertir_todo_a_minusculas" se conviertes los strings detectados  a minusculas en el dataframes

In [8]:
def convertir_todo_a_minusculas(data):
    """
    Convierte todos los strings en una estructura de datos a minúsculas.
    Puede manejar DataFrames, diccionarios, listas o combinaciones de estos.
    
    """
    
    # Caso 1: Es un DataFrame de pandas
    if isinstance(data, pd.DataFrame):
        try:
            df_procesado = data.copy()
            columnas_texto = df_procesado.select_dtypes(include=['object', 'string']).columns
            
            df_procesado[columnas_texto] = df_procesado[columnas_texto].apply(
                lambda x: x.str.lower() if pd.api.types.is_string_dtype(x) else x
            )
            
            print(f"✅ DataFrame procesado: {len(columnas_texto)} columnas de texto convertidas")
            return df_procesado
            
        except Exception as e:
            print(f"⚠ Error procesando DataFrame: {str(e)}")
            return data  # Devuelve el original si hay error
    
    # Caso 2: Es un diccionario
    elif isinstance(data, dict):
        resultado = {}
        for key, value in data.items():
            try:
                resultado[key] = convertir_todo_a_minusculas(value)
            except Exception as e:
                print(f"⚠ Error procesando clave {key}: {str(e)}")
                resultado[key] = value
        return resultado
    
    # Caso 3: Es una lista o tupla
    elif isinstance(data, (list, tuple)):
        try:
            return [convertir_todo_a_minusculas(item) for item in data]
        except Exception as e:
            print(f"⚠ Error procesando lista: {str(e)}")
            return data
    
    # Caso 4: Es un string
    elif isinstance(data, str):
        return data.lower()
    
    # Caso 5: Es un numpy array u otro tipo
    elif isinstance(data, np.ndarray):
        try:
            # Para arrays de strings
            if data.dtype.kind in ['U', 'S', 'O']:  # Unicode, string, object
                return np.char.lower(data)
            else:
                return data
        except Exception as e:
            print(f"⚠ Error procesando array numpy: {str(e)}")
            return data
    
    # Caso 6: Otros tipos (números, None, etc.)
    else:
        return data


# Ejemplo de uso:
dataframes = convertir_todo_a_minusculas(dataframes)

✅ DataFrame procesado: 10 columnas de texto convertidas
✅ DataFrame procesado: 4 columnas de texto convertidas
✅ DataFrame procesado: 7 columnas de texto convertidas
✅ DataFrame procesado: 10 columnas de texto convertidas
✅ DataFrame procesado: 3 columnas de texto convertidas
✅ DataFrame procesado: 19 columnas de texto convertidas


Con la funcion "convert_to_numeric" detecta en todas las tablas del diccionario las columnas que tengan valores que se puedan convertir a numeros y los convierte. 

In [9]:
def convert_to_numeric(df, min_numeric_ratio=0.8):
    """
    Convierte solo columnas con alto porcentaje de valores numéricos.
    
    Args:
        df: DataFrame a procesar.
        min_numeric_ratio: Mínimo de valores convertibles (0.8 = 80%) para aplicar cambios.
    """
    for col in df.columns:
        # Si la columna ya es numérica, saltar
        if pd.api.types.is_numeric_dtype(df[col]):
            continue
            
        # Intentar conversión
        converted = pd.to_numeric(df[col], errors='coerce')
        
        # Verificar ratio de éxito (ej: 80% de valores convertidos)
        success_ratio = 1 - converted.isna().mean()
        
        if success_ratio >= min_numeric_ratio:
            df[col] = converted
            
    return df


# Aplicar a todos los DataFrames (seguro y eficiente)
for name, df in dataframes.items():
    dataframes[name] = convert_to_numeric(df.copy())

En esta parte se eliminan durante el analisis las columnas indicadas sin que se modifique en los archivos.

In [10]:
#Se eliminaran columnas que no se ocuparan
columnas_a_eliminar = ['image', 'description', 'id', 'drops', 'drop', 'blockquote'	]  # Columnas redundantes
for nombre_tabla, df in dataframes.items():  
  dataframes[nombre_tabla] = df.drop(columns=columnas_a_eliminar, errors='ignore')

In [11]:
df_locations=dataframes['locations']

In [12]:
df_locations.head(1)

,location,region,items,npcs,creatures,bosses,dlc
0,abandoned ailing village,gravesite plain,"['Broken Rune', 'Golden Rune (1)', 'Black Pyrefly', 'Human Bone Shard', 'Fly Mold', 'Ailment Talisman', 'Revered Spirit Ash', ""Greater Potentate's Cookbook (10)"", 'Larval Tear']",['Spirit NPC'],['Man-Fly'],NaN,1
